In [14]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import *
import pymongo

In [15]:
sc = pyspark.SparkContext(appName="MongoDB injection")
spark = SparkSession(sc)

In [16]:
hdfs_path = "hdfs://localhost:54310/user/ubuntu/map_reduce/joint_dataset/"
schema = "Title string, Counter string, Authors string, Avg_Rating string, ISBN13 string, Num_Pages string, Ratings_Count string, Text_Reviews_Count string, Publisher string, Publication_Date string"

df = spark.read.csv(hdfs_path, header=False, inferSchema=False, sep="\t", quote='', escape='', schema=schema)

In [17]:
df_pandas = df.sample(1.0).toPandas()
df_pandas.head()

,Title,Counter,Authors,Avg_Rating,ISBN13,Num_Pages,Ratings_Count,Text_Reviews_Count,Publisher,Publication_Date
0,$30 film school: how to write direct produce...,1,Michael W. Dean,3.49,9781592000678,528,30,4,Cengage Learning,2003-05-13
1,1 000 places to see before you die,1,Patricia Schultz,3.85,9780761104841,992,36303,439,Workman Publishing Company,2003-05-22
2,10 lb penalty,1,Dick Francis,3.9,9780425197455,320,3490,177,G.P. Putnam's Sons,2004-08-03
3,100 great fantasy short short stories,1,Isaac Asimov/Terry Carr/Martin H. Greenberg/J...,3.91,9780380699179,395,204,19,Avon,1985-08-01
4,100 love sonnets,1,Pablo Neruda/Stephen Tapscott,4.39,9780292760288,232,12613,392,University of Texas Press,1986-01-01


In [18]:
my_client = pymongo.MongoClient("mongodb://localhost:27017/")
my_db = my_client["goodreads_db"]
my_collection = my_db["books"]
my_dictionary= df_pandas.to_dict(orient='records')

result = my_collection.insert_many(my_dictionary)

In [19]:
sc.stop()